<a href="https://colab.research.google.com/github/BunnyNoBugs/hse-headlines-project/blob/main/bart_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 6.0 MB/s eta 0:00:00
Looking in inde

In [ ]:
!wget https://raw.githubusercontent.com/BunnyNoBugs/hse-headlines-project/main/data/train.csv
!wget https://raw.githubusercontent.com/BunnyNoBugs/hse-headlines-project/main/data/test.csv

--2023-03-23 15:58:35--  https://raw.githubusercontent.com/BunnyNoBugs/hse-headlines-project/main/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 175396 (171K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 171.29K  --.-KB/s    in 0.003s  

2023-03-23 15:58:36 (57.8 MB/s) - ‘train.csv’ saved [175396/175396]

--2023-03-23 15:58:36--  https://raw.githubusercontent.com/BunnyNoBugs/hse-headlines-project/main/data/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19026 (19K) [t

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

from transformers import MBartTokenizer, MBartForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df = train_df[train_df['Type'] != 'Dishonest']
test_df = test_df[test_df['Type'] != 'Dishonest']
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
train_df["Type"].unique()

array(['Honest', 'Clickbait'], dtype=object)

In [ ]:
id2label = {0: "Honest", 1: "Clickbait"}
label2id = {"Honest": 0, "Clickbait": 1}

In [ ]:
#global constants

model_checkpoint = 'IlyaGusev/mbart_ru_sum_gazeta'
num_labels = train_df['Type'].nunique()

In [ ]:
tokenizer = MBartTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def preprocess_function(examples):
    tokenized_inputs = tokenizer(examples['Headline'], truncation=True, max_length=512, padding=True)
    tokenized_inputs['labels'] = [label2id[x] for x in examples['Type']]
    
    return tokenized_inputs

In [ ]:
encoded_train_dataset = train_dataset.map(preprocess_function, batched=True)
encoded_test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1165 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

In [ ]:
model = MBartForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at IlyaGusev/mbart_ru_sum_gazeta were not used when initializing MBartForSequenceClassification: ['lm_head.weight', 'final_logits_bias']
- This IS expected if you are initializing MBartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MBartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MBartForSequenceClassification were not initialized from the model checkpoint at IlyaGusev/mbart_ru_sum_gazeta and are newly initialized: ['classification_head.dense.weight', 'classification_head.dense.bias', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "steps",
    logging_steps=50,
    save_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=1,
    save_steps=500,
    # metric_for_best_model='f1_macro',
    # load_best_model_at_end=True
)

In [ ]:
def compute_metrics(p):
    outputs, labels = p
    predictions = np.argmax(outputs[0], axis=1)
    # predictions = predictions.reshape(-1, num_labels)

    # labels = labels.reshape(-1, num_labels)

    precision_macro = precision_score(labels, predictions, average='macro')
    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_classes = f1_score(labels, predictions, average=None)
    f1_classes = {f'f1_{cls_name}': f1 for cls_name, f1 in zip(label2id, f1_classes)}

    metrics = {
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
    }
    metrics.update(f1_classes)

    return metrics

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Precision Macro,Recall Macro,F1 Macro,F1 Honest,F1 Clickbait
50,0.600700,0.534116,0.390244,0.475248,0.428571,0.857143,0.000000
100,0.432000,0.619305,0.541980,0.524569,0.520817,0.846512,0.195122
150,0.379900,0.611502,0.590783,0.608361,0.595610,0.804124,0.387097
200,0.174800,0.879375,0.584842,0.582508,0.583604,0.827586,0.339623
250,0.091900,1.573089,0.584842,0.582508,0.583604,0.827586,0.339623
300,0.038800,1.545655,0.662281,0.594976,0.607941,0.874419,0.341463


KeyboardInterrupt: ignored